In [ ]:
# Import packages
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import yfinance as yf
import random
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Import modules
import trading_strategies

## Processing user input

Here you can enter parameters of your trading intentions, namely the period you want to observe the strategies performance in, the amount you intend to trade with, and the ticker you want to trade. Warning: Your initial balance must be greater than the close price of the ticker at the start of the period you picked (no fractional trades).

In [ ]:
# Specify the ticker:
ticker = input("Enter a ticker you want to trade with: ")

# Take user input for start date:
start_date_str = input("Enter the start date (YYYY-MM-DD): ")

# Take user input for end date:
end_date_str = input("Enter the end date (YYYY-MM-DD or 'today' if you want to the most recent data): ")

initial_balance = float(input("Enter the amount (in USD) you want to trade with: "))

# Check if the user entered 'today' for end date:
if end_date_str.lower() == 'today':
    end_date = datetime.today()
else:
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

# Convert user input date strings to datetime objects for start date:
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")

# Create a close price dataframe:
df = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    
# Check for missing values in the dataframe:
if df.isnull().values.any():
    print("Warning! Data for the requested ticker is not available for the whole period.")
else:
    print("Data downloaded successfully.")

# Trading Strategies

## Baseline strategies

### 1. Random Trading

Here you can see the plot of an approximate distribution of the results of the random trading method. You can also run a single simulation below to see a sample of how such strategy makes trades. It will describe the trades it made in your specified period along with the prices at which the trades were made.

In [ ]:
trading_strategies.random_trading(data = df, initial_balance = initial_balance, description = True)

In [ ]:
# Simulate trading cycles
trade_simulations = [trading_strategies.random_trading(data = df, initial_balance = initial_balance,
                                                       description = False) for i in range(1000)]

In [ ]:
# Plot the trade simulations distribution
plt.figure(figsize=(10,4))
sns.displot(trade_simulations)
plt.axvline(np.mean(trade_simulations), color='b', linestyle='-')
plt.axvline(100, color='g', linestyle='--')
plt.title('Avg: $%s\nSD: $%s'%(round(np.mean(trade_simulations),2), round(np.std(trade_simulations),2)), fontsize=20)

### 2. Buy and Hold

In [ ]:
trading_strategies.buy_and_hold(df, initial_balance)

## Data based strategies

### Moving Average 

In [ ]:
# create a dataframe for the Moving Average strategy
df_ma = trading_strategies.MA_strategy(data = df, long_ma = 50, short_ma = 20)

In [ ]:
# Visualization of Buy and sell signals using Moving Average
plt.figure(figsize = (16,8))
plt.title("Moving Average trade signals", fontsize = 18)
plt.plot(df_ma["Open"], alpha = 0.5)
plt.plot(df_ma["ShortMA"], alpha = 1)
plt.plot(df_ma["LongMA"], alpha = 0.5)
plt.scatter(df_ma.index, df_ma["Buy"], alpha = 1, label = "Buy Signal", marker = "^", color = "green")
plt.scatter(df_ma.index, df_ma["Sell"], alpha = 1, label = "Sell Signal", marker = "v", color = "red")
plt.xlabel("Date", fontsize = 18)
plt.ylabel("Open price", fontsize = 18)
plt.show

### XGBoost Regression 

In [ ]:
df_trading = trading_strategies.xgb_signals(df)

In [ ]:
# Plot buy and sell signals
buy_signals = df_trading[df_trading['Trade_Signal'] == 1]
sell_signals = df_trading[df_trading['Trade_Signal'] == -1]

# Create a wider figure
plt.figure(figsize=(15, 8))

# Scatter plots for buy and sell signals
plt.scatter(buy_signals.index, buy_signals['Close_Price_On_Trade'], marker='^', color='g', label='Buy Signal')
plt.scatter(sell_signals.index, sell_signals['Close_Price_On_Trade'], marker='v', color='r', label='Sell Signal')

# Customize the plot
plt.title('Buy and Sell Signals with Price Line')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()